# Генерация синтетических данных с использованием Faker

In [1]:
!pip install faker


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

In [3]:
# Инициализация генератора (русская локаль для реалистичных ФИО, адресов и т.д.)
fake = Faker('ru_RU')
Faker.seed(42)      # фиксируем seed для воспроизводимости
random.seed(42)

## Генерация синтетических данных для варианта "Сотрудники компании"

In [4]:
# DataFrame с отделами
def generate_departments(n):
   
    departments = []
    for i in range(1, n + 1):
        departments.append({
            'department_id': i,
            'name': fake.unique.job() + ' отдел',  # уникальное название
            'location': fake.city()
        })
    return pd.DataFrame(departments)

# DataFrame с должностями и диапазоном зарплат
def generate_positions(n: int) -> pd.DataFrame:
    
    positions = []
    for i in range(1, n + 1):
        min_salary = random.randint(30000, 60000)
        max_salary = min_salary + random.randint(20000, 80000)
        positions.append({
            'position_id': i,
            'name': fake.unique.job(),
            'min_salary': min_salary,
            'max_salary': max_salary
        })
    return pd.DataFrame(positions)

# генерируем сотрудников со случайными атрибутами
def generate_employees(n: int, department_ids: list, position_ids: list) -> pd.DataFrame:
    
    employees = []
    for emp_id in range(1, n + 1):        
        first_name = fake.first_name()
        last_name = fake.last_name()
        patronymic = fake.middle_name()
        birth_date = fake.date_of_birth(minimum_age=18, maximum_age=70)
        phone = fake.phone_number()
        email = fake.email()
        address = fake.address().replace('\n', ', ')
        
        # дата найма: не раньше 18-летия и не позже сегодняшнего дня
        min_hire_date = birth_date + timedelta(days=18*365)
        hire_date = fake.date_between(start_date=min_hire_date, end_date='today')
        
        # статус -  active / terminated (10% уволены)
        status = random.choices(['active', 'terminated'], weights=[0.9, 0.1])[0]
        
        # выбор отдела и должности
        department_id = random.choice(department_ids)
        position_id = random.choice(position_ids)
        
        # зарплата     
        salary = random.randint(40000, 150000)
        
        employees.append({
            'employee_id': emp_id,
            'last_name': last_name,
            'first_name': first_name,
            'patronymic': patronymic,
            'birth_date': birth_date,
            'gender': random.choice(['М', 'Ж']),
            'address': address,
            'phone': phone,
            'email': email,
            'hire_date': hire_date,
            'status': status,
            'department_id': department_id,
            'position_id': position_id,
            'salary': salary
        })
    return pd.DataFrame(employees)

# создать историю зарплат на основе данных сотрудников
def generate_salary_history(employees_df: pd.DataFrame, avg_records: int = 3):    
    history = []
    record_id = 1
    
    for _, emp in employees_df.iterrows():
        emp_id = emp['employee_id']
        hire_date = emp['hire_date']
        current_salary = emp['salary']
        
        # количество изменений (0 – если сотрудник только нанят и ещё не было изменений)
        num_changes = random.choices([0, 1, 2, 3, 4], weights=[0.2, 0.3, 0.3, 0.1, 0.1])[0]
        
        # генерируем даты изменений (после hire_date и до сегодня)
        change_dates = sorted([fake.date_between(start_date=hire_date, end_date='today') 
                               for _ in range(num_changes)])
        
        # начальная зарплата при найме (можно сделать немного отличающейся от текущей)
        #  будем считать, что первая запись – это зарплата при найме,
        # а последующие – повышения.
        if num_changes == 0:
            # Если изменений не было, всё равно добавим одну запись (начальная)
            history.append({
                'history_id': record_id,
                'employee_id': emp_id,
                'change_date': hire_date,
                'new_salary': current_salary
            })
            record_id += 1
        else:
            # генерируем возрастающие зарплаты
            salary_values = sorted([random.randint(30000, current_salary) for _ in range(num_changes)])
            # добавляем текущую зарплату как последнюю
            salary_values.append(current_salary)
            # даты: hire_date и change_dates
            all_dates = [hire_date] + change_dates
            for i in range(len(all_dates)):
                history.append({
                    'history_id': record_id,
                    'employee_id': emp_id,
                    'change_date': all_dates[i],
                    'new_salary': salary_values[i]
                })
                record_id += 1
    return pd.DataFrame(history)

In [5]:
N_DEPARTMENTS = 10
N_POSITIONS = 20
N_EMPLOYEES = 500

In [6]:
departments_df = generate_departments(N_DEPARTMENTS)
positions_df = generate_positions(N_POSITIONS)
employees_df = generate_employees(
    N_EMPLOYEES,
    departments_df['department_id'].tolist(),
    positions_df['position_id'].tolist()
)

In [7]:
departments_df

,department_id,name,location
0,1,Дефектолог отдел,клх Кырен
1,2,Машинистка отдел,п. Волоколамск
2,3,Географ отдел,г. Артем
3,4,Гирудотерапевт отдел,п. Териберка
4,5,Бактериолог отдел,клх Усинск
5,6,Стоматолог отдел,с. Чикола
6,7,Нарколог отдел,ст. Ессентуки
7,8,Стрелочник отдел,к. Ельня
8,9,Крупье отдел,г. Видное
9,10,Рефлексотерапевт отдел,к. Нефедова


In [8]:
positions_df

,position_id,name,min_salary,max_salary
0,1,Модельер,50952,78248
1,2,Борт-радист,30819,99417
2,3,Травматолог,39012,75061
3,4,Диспетчер,37314,66458
4,5,Рентгенолог,54132,80849
5,6,Врач МСЭК,52174,120714
6,7,Оперативный работник,59234,114975
7,8,Радиолог,32848,91546
8,9,Кодер,43825,65907
9,10,Военный полицейский,30976,57116


In [9]:
employees_df

,employee_id,last_name,first_name,patronymic,birth_date,gender,address,phone,email,hire_date,status,department_id,position_id,salary
0,1,Гаврилов,Мина,Кузьминична,1973-04-17,М,"к. Бологое, пер. Островского, д. 77, 242388",+7 928 327 64 83,tatjana_03@example.org,2009-02-27,active,7,11,76421
1,2,Григорьева,Ольга,Федотович,1994-11-01,Ж,"д. Красногорск (Моск.), ш. Заречное, д. 684 ст...",8 532 871 01 22,erjabova@example.com,2023-06-17,active,6,4,52156
2,3,Крюкова,Лаврентий,Егоровна,1989-09-03,М,"п. Хасавюрт, ул. Вахитова, д. 382, 824896",83252880957,makarovamaja@example.com,2014-11-13,active,6,20,74671
3,4,Лукина,Тимур,Николаевна,2002-01-26,М,"г. Яхрома, алл. Профсоюзная, д. 951, 473829",+7 (465) 787-1331,adrian_98@example.org,2024-12-16,active,9,4,89615
4,5,Мартынов,Савватий,Станиславовна,1987-04-24,М,"п. Уварово, алл. Челюскинцев, д. 84 к. 81, 080132",8 311 656 67 01,maslovalukija@example.org,2019-02-15,active,10,12,115674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Фомина,Амвросий,Валерьевич,1966-07-21,Ж,"клх Котельнич, алл. Угольная, д. 4/4 стр. 356,...",+78255442867,ikarpov@example.com,1996-08-23,active,4,16,112173
496,497,Гуляев,Мефодий,Владленович,1994-04-14,М,"к. Ардон, пр. Механический, д. 9/6 стр. 895, 4...",+7 (693) 358-95-13,vatslav_2008@example.net,2014-04-24,active,8,16,100401
497,498,Корнилова,Евдокия,Якубович,1994-05-22,Ж,"п. Партизанск, ул. 40 лет Победы, д. 8 к. 455,...",+7 179 720 38 59,karpevseev@example.org,2020-06-20,active,6,6,58344
498,499,Терентьева,Милица,Андреевна,1961-10-02,М,"д. Тура, бул. Советской Армии, д. 892, 452388",+7 (675) 009-4214,petrovmina@example.org,2017-09-12,active,9,2,108674


In [10]:
# Генерация истории зарплат
salary_history_df = generate_salary_history(employees_df, avg_records=3)
salary_history_df

,history_id,employee_id,change_date,new_salary
0,1,1,2009-02-27,71333
1,2,1,2022-12-24,76421
2,3,2,2023-06-17,34948
3,4,2,2025-04-12,52156
4,5,3,2014-11-13,74671
...,...,...,...,...
1291,1292,498,2021-09-18,57426
1292,1293,498,2024-04-13,58344
1293,1294,499,2017-09-12,89827
1294,1295,499,2018-05-29,108674


In [11]:
# Сохранение в CSV
departments_df.to_csv('departments.csv', index=False)
positions_df.to_csv('positions.csv', index=False)
employees_df.to_csv('employees.csv', index=False)
salary_history_df.to_csv('salary_history.csv', index=False)

# Задание на самостоятельную работу
Создать синтетический набор данных, сохранить его в csv файлы, построить на основе данных онтологию, вывести онтограф.


### Вариант 1. Аренда электромобилей
Сущности: 
- Автомобиль: id, марка, модель, год выпуска, госномер, запас хода (км), статус (доступен, арендован, на обслуживании), тариф (цена за минуту), локация (парковка). История зарядок (JSON-массив объектов: {id станции, дата_начала, дата_конца, объём_кВт·ч, стоимость}).
- Клиент: id, ФИО, водительское удостоверение (серия, номер), телефон, email, дата регистрации.
- Аренда:id, id клиента, id автомобиля, дата и время начала, дата и время окончания, начальный пробег, конечный пробег, стоимость, статус (активна, завершена, отменена), Информация об оплате (поля внутри таблицы): сумма, способ оплаты, дата оплаты, статус оплаты.
- Зарядная станция: id, адрес, тип разъёма, количество мест, статус (работает/не работает).

Объём данных: автомобили - 100, клиенты - 500, аренды - 3000, зарядные станции -  20

Онтология:
- Классы: Автомобиль, Клиент, Аренда, ЗаряднаяСтанция.
- Связи:  клиент арендует автомобиль, автомобиль участвует в арендах, история зарядок хранится в JSON внутри автомобиля, ссылаясь на станции, Аренда содержит данные об оплате.

### Вариант 2. Банковские клиенты и транзакции
Сущности: 
- Клиент: id, ФИО, дата рождения, паспортные данные (серия, номер, кем выдан), адрес, телефон, email.
- Счёт: id, номер счёта, тип (дебетовый/кредитный), валюта, дата открытия, остаток, id клиента.
- Транзакция: id, дата, сумма, тип (пополнение/списание), id счёта, описание.
- Кредит: id, id клиента, сумма, процентная ставка, срок, дата выдачи, ежемесячный платёж.

Объём: 1000 клиентов, у каждого 1–3 счёта, на каждом 10–50 транзакций, 20% клиентов имеют кредиты.

Онтология:
- Классы: Клиент, Счёт, Транзакция, Кредит.
- Связи: клиент владеет счетами, по счетам проходят транзакции, клиент может иметь кредиты.

### Вариант 3. Медицинские пациенты и приёмы
Сущности:
- Пациент: id, ФИО, дата рождения, пол, полис ОМС, СНИЛС, адрес, телефон.
- Врач: id, ФИО, специальность, кабинет.
- Приём: id, дата и время, id пациента, id врача, диагноз (код МКБ-10), жалобы, назначения (JSON-массив объектов: {лекарство, дозировка, курс}).
- Больничный лист: id, id приёма, дата начала, дата окончания, диагноз.

Объём: 2000 пациентов, 50 врачей, 5–10 приёмов на пациента за последние 2 года, часть с больничными.

Онтология:
- Классы: Пациент, Врач, Приём, БольничныйЛист.
- Связи: пациент посещает врача (приём), по приёму может быть открыт больничный, назначения хранятся в JSON приёма (Атрибут класса Прием).

### Вариант 4. Студенты и успеваемость
Сущности:
- Студент: id, ФИО, дата рождения, группа, год поступления, форма обучения.
- Группа: id, номер, курс, институт, направление подготовки.
- Предмет: id, название, семестр, количество часов, форма контроля (экзамен/зачёт), id преподавателя.
- Оценка: id, id студента, id предмета, дата, оценка (число или зачёт/незачёт).
- Преподаватель: id, ФИО, кафедра.

Объём: 10 групп по 25 студентов = 250 студентов, 30 предметов, у каждого студента оценки по всем предметам (≈ 7500 записей).

Онтология
- Классы: Студент, Группа, Предмет, Оценка, Преподаватель.
- Связи: студент учится в группе, группа изучает предметы (связка группа-предмет), преподаватель ведёт предмет, студент получает оценки.

### Вариант 5. Интернет-магазин: товары, заказы, покупатели
Сущности:
- Покупатель: id, email, ФИО, телефон, дата регистрации.
- Товар: id, название, категория, цена, остаток на складе.
- Заказ: id, id покупателя, дата, статус (новый, оплачен, отгружен, доставлен).
- Состав заказа: id заказа, id товара, количество, цена в момент заказа.
- Отзыв: id, id покупателя, id товара, оценка (1–5), текст, дата.

Объём: 500 покупателей, 1000 товаров, 3000 заказов (в среднем по 2 товара в заказе), 1500 отзывов.

Онтология:
- Классы: Покупатель, Товар, Заказ, СтрокаЗаказа, Отзыв.
- Связи: покупатель делает заказ, заказ содержит товары (с количеством), покупатель оставляет отзыв на товар.

### Вариант 6. Недвижимость и сделки
Сущности: 
- Объект недвижимости: id, адрес, тип (квартира/дом/коммерческое), площадь, количество комнат, этаж, год постройки, кадастровый номер.
- Владелец: id, ФИО, паспортные данные, доля (если долевая собственность).
- Сделка купли-продажи: id, дата, id объекта, id продавца, id покупателя, цена, нотариус.
- Ценовой архив: id объекта, дата изменения, новая цена (для отслеживания динамики).

Объём: 2000 объектов, 300 владельцев, 150 сделок, у каждого объекта 2–3 записи цен.

Онтология:
- Классы: Объект, Владелец, Сделка, ЦеновойАрхив.
- Связи: объект может иметь нескольких владельцев (доли), сделка связывает продавца, покупателя и объект.

### Вариант 7. Транспортные средства и штрафы
Сущности:
- Владелец ТС: id, ФИО, водительское удостоверение (серия, номер), адрес, телефон.
- Автомобиль: id, госномер, марка, модель, год выпуска, VIN, цвет, id владельца.
- Штраф: id, id автомобиля, дата, статья нарушения, сумма, статус (оплачен/не оплачен).
- Страховой полис: id, id автомобиля, компания, дата начала, дата окончания, стоимость.

Объём: 1000 владельцев, 1200 автомобилей (некоторые владеют несколькими), 5000 штрафов, 1500 полисов.

Онтология:
- Классы: Владелец, Автомобиль, Штраф, Полис.
- Связи: владелец владеет автомобилями, автомобиль имеет штрафы и страховки.

### Вариант 8. Библиотека: книги, читатели, выдачи
Сущности:
- Книга: id, ISBN, название, автор, год издания, издательство, жанр, количество экземпляров в библиотеке.
- Читатель: id, ФИО, дата рождения, адрес, телефон, email, дата регистрации.
- Выдача: id, id книги, id читателя, дата выдачи, дата возврата (если есть), фактическая дата возврата.
- Бронь: id, id книги, id читателя, дата брони, статус (активна/исполнена).

Объём: 500 книг, 300 читателей, 2000 выдач, 100 броней.

Онтология:
- Классы: Книга, Читатель, Выдача, Бронь.
- Связи: читатель берёт книгу (выдача), может бронировать.

### Вариант 9. Техподдержка: заявки и обращения
Сущности:
- Пользователь: id, ФИО, email, телефон, уровень (обычный/премиум).
- Категория заявки: id, название, описание.
- Сотрудник поддержки: id, ФИО, отдел, количество решённых заявок.
- Заявка: id, id пользователя, id категории, тема, описание, дата создания, статус (новая, в работе, решена, закрыта), приоритет, id сотрудника (назначен), комментарии (JSON-массив объектов: {id автора (пользователь или сотрудник), текст, дата}).

Объём данных: пользователи - 1000, категории - 5, сотрудники - 10, заявки - 5000, комментарии - 15 000  (общее на все заявки)

Онтология:
- Классы: Пользователь, Категория, Сотрудник, Заявка.
- Связи: пользователь создаёт заявку; заявка относится к категории; заявка назначается сотруднику; комментарии принадлежат заявке (в JSON).

### Вариант 10. Доставка обедов из ресторанов
Сущности: 
- Ресторан: id, название, адрес, кухня (тип кухни), рейтинг.
- Блюдо: id, название, id ресторана, категория, цена, вес/порция, калории.
- Клиент: id, ФИО, телефон, email, адрес доставки (основной), дата регистрации.
- Заказ: id, id клиента, id ресторана, дата и время заказа, статус (принят, готовится, доставлен, отменён), общая сумма, способ оплаты, состав заказа (JSON-массив объектов: {id блюда, количество, цена_в_момент_заказа}), информация о доставке (поля внутри той же таблицы): курьер (ФИО), дата и время назначения курьера, дата и время фактической доставки, статус доставки (назначен, в пути, выполнен), фактический адрес доставки (если отличается от адреса клиента)

Объём данных: рестораны -  20, блюда - в среднем 20 на ресторан, клиенты -  1000, заказы - 5000.

Онтология: 
- Классы: Ресторан, Блюдо, Клиент, Заказ.
- Связи: ресторан предлагает блюда, клиент оформляет заказ, заказ ссылается на ресторан и клиента.

Состав заказа и данные доставки хранятся внутри сущности «Заказ» (денормализовано с использованием JSON).
